<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/AMPDs/training/nilm_rnn_cwe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [0]:
WHE_data = pd.read_csv('Electricity_WHE.csv')['P']
CWE_data = pd.read_csv('Electricity_CWE.csv')['P']


In [0]:
window_size=60

dataX_raw = create_dataset(WHE_data.as_matrix(), window_size)

cwe_Y_raw = CWE_data[window_size-1:].values.reshape([CWE_data.shape[0]-window_size+1,1])

dataX = np.concatenate([dataX_raw[0:472500,:],dataX_raw[475500:,:]],axis=0)
cwe_Y = np.concatenate([cwe_Y_raw[0:472500,:],cwe_Y_raw[475500:,:]],axis=0)
categorized_cwe_Y = np.ones(cwe_Y.shape)*2
categorized_cwe_Y[[np.where(cwe_Y==0)[0]],:]=0
categorized_cwe_Y[[np.where((cwe_Y>0)&(cwe_Y<=350))[0]],:]=1
#categorized_cwe_Y[[np.where((cwe_Y>300))[0]],:]=2

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cweY_1hot = encoder.fit_transform(categorized_cwe_Y)

X_train, X_test, y_train, y_test = train_test_split(dataX, cweY_1hot, test_size=0.01, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
max_indice = np.argmax(cweY_1hot,1)
df = pd.Series(max_indice)
df.value_counts()

0    1018321
1      26515
2       3305
dtype: int64

In [0]:
# Hyper Parameters
LR = 0.002      # learning rate
batch_size=2048
window_size=60
num_units = [128, 64]

'''RNN Model Definition'''
tf.reset_default_graph()
''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1],name='x')
tf_y = tf.placeholder(tf.int32, [None, 3],name='y')


cells = [tf.keras.layers.LSTMCell(units=n) for n in num_units]
stacked_rnn_cell = tf.keras.layers.StackedRNNCells(cells)
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
        stacked_rnn_cell,                   # cell you have chosen
        tf_x,                      # input
        initial_state=None,         # the initial hidden state
        dtype=tf.float32,           # must given if set initial_state = None
        time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)

l1 = tf.layers.dense(outputs[:, -1, :],64,activation=tf.nn.relu,name='l1',\
                    kernel_initializer=tf.contrib.layers.xavier_initializer(),\
                    bias_initializer=tf.contrib.layers.xavier_initializer())\
                    #kernel_regularizer=tf.keras.regularizers.l2(0.001))
l2 = tf.layers.dense(l1,32,activation=tf.nn.relu,name='l2',\
                    kernel_initializer=tf.contrib.layers.xavier_initializer(),
                    bias_initializer=tf.contrib.layers.xavier_initializer())
l3 = tf.layers.dense(l2,16,activation=tf.nn.relu,name='l3'\
                    ,kernel_initializer=tf.contrib.layers.xavier_initializer(),
                    bias_initializer=tf.contrib.layers.xavier_initializer())
l4 = tf.layers.dense(l3,8,activation=tf.nn.leaky_relu,name='l4')
#l5 = tf.layers.dense(l4,256,activation=tf.nn.leaky_relu,name='l5')
#l6 = tf.layers.dense(l5,128,activation=tf.nn.leaky_relu,name='l6')
#l7 = tf.layers.dense(l6,84,activation=tf.nn.leaky_relu,name='l7')
#l8 = tf.layers.dense(l7,72,activation=tf.nn.leaky_relu,name='l8')
#l9 = tf.layers.dense(l8,64,activation=tf.nn.leaky_relu,name='l9')
#l10 = tf.layers.dense(l9,32,activation=tf.nn.leaky_relu,name='l10')
#l11 = tf.layers.dense(l10,16,activation=tf.nn.leaky_relu,name='l11')
#l12 = tf.layers.dense(l11,8,activation=tf.nn.leaky_relu,name='l12')
#l13 = tf.layers.dense(l12,4,activation=tf.nn.leaky_relu,name='l13')
pred = tf.layers.dense(l3,3,activation=tf.nn.relu,name='pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()

In [20]:
from imblearn.under_sampling import RandomUnderSampler
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
tl = RandomUnderSampler(sampling_strategy={0:3000,1:3000,2:3000})
#from imblearn.under_sampling import EditedNearestNeighbours
#tl = EditedNearestNeighbours(sampling_strategy={0:5000,1:5000})
#print('finish t1')
sess = tf.Session()
sess.run(init_op)
#saver.restore(sess, 'my_net_cwe/save_cwe_rnn.ckpt')
for i in range(0,10):
  X, y = tl.fit_resample(X_train, y_train)
  #y = encoder.fit_transform(y.reshape([-1,1]))
  whole_train = np.concatenate([X,y],axis=1)
  batch_index = np.random.choice(9000,batch_size)
  batch_train = whole_train[batch_index,:]
  batch_X = batch_train[:,:-3].reshape([batch_size,window_size,1])
  batch_y = batch_train[:,-3:]
  for j in range(0,100):
      print('##·##Loop:'+str(i)+'###iteration: '+str(j)+'###')
      sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
      cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
      acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
      acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), \
                                              tf_y:y_test})
      print('train loss= %.16f' % cost_+', Acc=%.4f'% acc_train)
      print('Test Acc=%.2f'% acc_test)

      pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
      y_lables_argmax = tf.argmax(tf_y,axis=1)  
      y_pred_argmax = tf.argmax(pre,axis=1)
      confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
      #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
      print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
      if(j//99==0):
        save_path = saver.save(sess, "my_net_cwe/save_cwe_rnn.ckpt")

      
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()

##·##Loop:0###iteration: 0###
train loss= 1.1250319480895996, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 1###
train loss= 1.1198840141296387, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 2###
train loss= 1.1152625083923340, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 3###
train loss= 1.1108379364013672, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 4###
train loss= 1.1066274642944336, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 5###
train loss= 1.1030609607696533, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 6###
train loss= 1.1002762317657471, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loop:0###iteration: 7###
train loss= 1.0993764400482178, Acc=0.3828
Test Acc=0.00
[[ 0  0 78]
 [ 0  0 80]
 [ 0  0 98]]
##·##Loo

KeyboardInterrupt: ignored

In [24]:
###############new####################
from imblearn.under_sampling import RandomUnderSampler
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
tl = RandomUnderSampler(sampling_strategy={0:3000,1:3000,2:3000})
sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net_cwe/save_cwe_rnn.ckpt')
batchX_list=[]
batchY_list=[]
X, y = tl.fit_resample(X_train, y_train)
whole_train = np.concatenate([X,y],axis=1)
for i in range(0,1):
  batch_index = np.random.choice(9000,batch_size)
  batch_train = whole_train[batch_index,:]
  batch_X = batch_train[:,:-3].reshape([batch_size,window_size,1])
  batch_y = batch_train[:,-3:]
  for j in range(0,100):
      print('##·##Loop:'+str(i)+'###iteration: '+str(j)+'###')
      sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
      cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
      acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
      acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), \
                                              tf_y:y_test})
      print('train loss= %.16f' % cost_+', Acc=%.4f'% acc_train)
      print('Test Acc=%.2f'% acc_test)

      pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_y})
      y_lables_argmax = tf.argmax(tf_y,axis=1)  
      y_pred_argmax = tf.argmax(pre,axis=1)
      confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
      #print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
      print(confusion.eval(session=sess,feed_dict={tf_x: batch_X, tf_y:batch_y}))
      if(j//9==0):
        save_path = saver.save(sess, "my_net_cwe/save_cwe_rnn.ckpt")

      
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()

INFO:tensorflow:Restoring parameters from my_net_cwe/save_cwe_rnn.ckpt
##·##Loop:0###iteration: 0###
train loss= 0.8495635390281677, Acc=0.6143
Test Acc=0.44
[[285 262 111]
 [ 52 461 184]
 [  4 177 512]]
##·##Loop:0###iteration: 1###
train loss= 0.8358542919158936, Acc=0.6230
Test Acc=0.45
[[296 236 126]
 [ 53 442 202]
 [  3 152 538]]
##·##Loop:0###iteration: 2###
train loss= 0.8259117007255554, Acc=0.6152
Test Acc=0.46
[[303 215 140]
 [ 67 395 235]
 [  3 128 562]]
##·##Loop:0###iteration: 3###
train loss= 0.8235347270965576, Acc=0.6172
Test Acc=0.47
[[315 192 151]
 [ 83 362 252]
 [  5 101 587]]
##·##Loop:0###iteration: 4###
train loss= 0.8239102363586426, Acc=0.6104
Test Acc=0.48
[[317 184 157]
 [ 91 346 260]
 [  8  98 587]]
##·##Loop:0###iteration: 5###
train loss= 0.8195563554763794, Acc=0.6143
Test Acc=0.47
[[314 179 165]
 [ 81 349 267]
 [  6  92 595]]
##·##Loop:0###iteration: 6###
train loss= 0.8111745119094849, Acc=0.6226
Test Acc=0.46
[[305 192 161]
 [ 69 374 254]
 [  3  94 596]

KeyboardInterrupt: ignored

In [15]:

!mkdir my_net_cwe


mkdir: cannot create directory ‘my_net_cwe’: File exists


In [0]:
rm -r my_net_cwe

In [0]:
mv checkpoint save_cwe_rnn.ckpt.data-00000-of-00001 save_cwe_rnn.ckpt.index save_cwe_rnn.ckpt.meta my_net_cwe

In [0]:
rm tomekLinked_data